### Notebook for the differential abundance analysis on mixed cell states using `milopy`

- **Developed by:** Carlos Talavera-López Ph.D
- **Institute of Systems Immunology (WüSI) - JMU-Würzburg**
- v230730

### Import required packages

In [1]:
import anndata
import numpy as np
import pandas as pd
import scanpy as sc 
import milopy.core as milo
import milopy.plot as milopl
import matplotlib.pyplot as plt

### Set up working environment

In [2]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

-----
anndata     0.9.2
scanpy      1.9.3
-----
PIL                 10.0.0
appnope             0.1.3
asttokens           NA
backcall            0.2.0
cffi                1.15.1
comm                0.1.3
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.2
debugpy             1.6.7
decorator           5.1.1
executing           1.2.0
h5py                3.9.0
importlib_resources NA
ipykernel           6.25.0
ipywidgets          8.0.7
jedi                0.19.0
jinja2              3.1.2
joblib              1.3.1
kiwisolver          1.4.4
llvmlite            0.40.1
markupsafe          2.1.3
matplotlib          3.7.2
milopy              0.1.1
mpl_toolkits        NA
natsort             8.4.0
numba               0.57.1
numpy               1.24.4
packaging           23.1
pandas              2.0.3
parso               0.8.3
patsy               0.5.3
pexpect             4.8.0
pickleshare         0.7.5
pkg_resources       NA
platformdirs        3.10.0
prompt_toolkit      3.0

### Read in data

In [ ]:
adata = sc.read('../../../data/Epithelial_Mixed_states_locked_ctl230730.raw.h5ad')
adata

### Visualise manifold

In [ ]:
sc.pl.umap(adata, frameon = False, color = ['IAV_score', 'cell_states'], size = 3, legend_fontsize = 8, ncols = 3)